

```
## Treasure Drop Game
  1. Muhamad Iqbal - G1A021073
  2. Fajar Adhitia Suwandhi - G1A021086
  3. Irfan Luthfi - G1A021090
```



In [1]:
import numpy as np
from collections import defaultdict

In [2]:
class Lever:
    def __init__(self):
        self.position = np.random.randint(low=0, high=2)
        self.coin = 0
        self.leftChild = None  #these are either levers or ints.
        self.rightChild = None
        self.leftMouth = 0
        self.rightMouth = 0
    def printTop(self):  #prints top part and puts a | character end of it.
        letter = 'V' if self.coin == 0 else 'O'
        if self.position == 0:  #means left
            print(letter + '  |', end="")
        else:
            print('  ' + letter + '|', end="")
    def printMiddle(self):
        if self.position == 0:
            print(' \ |', end="")
        else:
            print(' / |', end="")
    def printBottom(self):
        if self.position == 0:
            print('  \ ', end="")
        else:
            print('/   ', end="")

    def spitToLeft(self, i):
        if self.leftChild == None: #for testing purposes
            return 0
        elif type(self.leftChild) == int:  #It's left child is a bucket therefore it's returning a score.
            return i*self.leftChild
        else:
            self.leftChild.rightMouth += i
            return 0

    def spitToRight(self, i):
        if self.rightChild == None: #for testing purposes
            return 0
        if type(self.rightChild) == int:
            return i*self.rightChild
        else:
            self.rightChild.leftMouth += i
            return 0
    def applyMotion(self):
        score = 0

        if self.coin == 1:  #CASE 1
            temp = self.leftMouth + self.rightMouth
            self.leftMouth = 0
            self.rightMouth = 0

            if self.position == 0:  #O--
                score += self.spitToRight(temp)
                self.leftMouth = 1
                self.coin = 0 #coin airborne
            else:
                score += self.spitToLeft(temp)
                self.rightMouth = 1
                self.coin = 0 #coin airborne

            if temp % 2 == 1:  #sum of coins is odd. Let's flip the lever
                self.position = 0 if self.position == 1 else 1

        elif self.coin == 0:  #CASE 2, 3 and 4
            #Here, the order of if statements are important. Do not change the order.
            if self.leftMouth != 0 and self.rightMouth != 0 : #CASE 4
                temp = self.leftMouth + self.rightMouth - 1
                self.leftMouth = 0
                self.rightMouth = 0

                if self.position == 0:  #U--
                    score += self.spitToRight(temp)
                    self.leftMouth = 1  #coin airborne
                else:  #--U
                    score += self.spitToLeft(temp)
                    self.rightMouth = 1  #coin airborne

                if temp % 2 == 1:  #sum of coins is odd. Let's flip the lever.
                    self.position = 0 if self.position == 1 else 1

            elif (self.leftMouth != 0 and self.position == 0) or (self.rightMouth != 0 and self.position == 1):
                #CASE 2: Coin only arrives on top of empty slot.

                temp = self.leftMouth + self.rightMouth
                self.leftMouth = 0
                self.rightMouth = 0
                self.coin = 1
                temp -= 1

                if temp>0:  #Apparently, more than one coin arrived to the empty slot.
                    if self.position == 0:  #U--
                        score += self.spitToRight(temp)
                        self.leftMouth = 1  #coin airborne
                        self.coin = 0
                    else:  #--U
                        score += self.spitToLeft(temp)
                        self.rightMouth = 1  #coin airborne
                        self.coin = 0

                if temp % 2 == 1:
                    self.position = 0 if self.position == 1 else 1

            elif (self.leftMouth != 0 and self.position == 1) or (self.rightMouth != 0 and self.position == 0):
                #CASE 3: Coin only arrives to the non-slot side of lever.

                temp = self.leftMouth + self.rightMouth
                self.leftMouth = 0
                self.rightMouth = 0

                if self.position == 0:  #U--
                    score += self.spitToRight(temp)
                else:  #--U
                    score += self.spitToLeft(temp)

                if temp % 2 == 1:
                    self.position = 0 if self.position == 1 else 1

        return score

In [3]:
def fib(n):
    if n==1:
        return 1
    elif n==2:
        return 2
    else:
        return fib(n-1)+fib(n-2)

def fibonacciBuckets(length):  #length must be the number of lever in the bottom row.
    buckets = [fib(n+1) for n in range(length)]
    bucketsR = buckets.copy()
    bucketsR.reverse()
    return bucketsR+buckets

In [4]:
class TreasureDrop:
    def __init__(self, w=4, h=5, player=1, goalScore = 100):
        #init levers bottom up, starting from buckets
        if player != 1 and player != 2:  #player is who goes first here.
            raise(Exception("Starting player can be 1 or 2, 1 by defauly."))
        self.player = player
        self.winner = None
        self.isOver = False
        self.p1score = 0
        self.p2score = 0
        self.goalScore = goalScore
        self.w = w
        self.h = h
        self.buckets = fibonacciBuckets(w+self.h-1)  #This should be ints
        self.levers = [[Lever() for n in range(w+i)] for i in range(self.h)]
        #Set the lever hierarchy
        for rowIndex in range(len(self.levers)):
            for colIndex in range(len(self.levers[rowIndex])):
                if rowIndex < len(self.levers)-1:  #Not the last row of levers
                    self.levers[rowIndex][colIndex].leftChild = self.levers[rowIndex+1][colIndex]
                    self.levers[rowIndex][colIndex].rightChild = self.levers[rowIndex+1][colIndex+1]
                else:  #The last row of levers, buckets should be their children.
                    self.levers[rowIndex][colIndex].leftChild = self.buckets[colIndex*2]
                    self.levers[rowIndex][colIndex].rightChild = self.buckets[colIndex*2+1]

    def dropCoin(self, loc, verbose=False):  #loc starts from zero.
        #Exception if game already ended or invalid location for coin
        if self.winner:
            raise(Exception("There is a winner. Can't play anymore."))
        elif loc >= self.w*2:
            raise(Exception("Invalid loc for coin"))

        #Put the coin in proper mouth.
        colIndex = loc//2
        if loc%2 == 0:  #give it to left mouth
            self.levers[0][colIndex].leftMouth = 1
        elif loc%2 == 1:  #give it to right mouth
            self.levers[0][colIndex].rightMouth = 1

        #Loop through levers until all mouths are empty
        #Give every lever a timestep to move.
        #Also, Collect the score at the same time
        allMouthsAreEmpty = False
        score = 0
        while not allMouthsAreEmpty:  #Update should be from bottom rows all the way up
            allMouthsAreEmpty = True
            for r in range(len(self.levers)-1, -1, -1):  #Going through levers in a reverse order.
                for i, l in enumerate(self.levers[r]):  #enumerating for debugging purposes.
                    if l.leftMouth != 0 or l.rightMouth != 0:
                        allMouthsAreEmpty = False
                        score += l.applyMotion()

        #Give score to player
        if self.player == 1:
            self.p1score += score
        elif self.player == 2:
            self.p2score += score

        #It's other player's turn
        if self.player == 1:
            self.player = 2
        elif self.player == 2:
            self.player = 1

        #Check if the game ends
        if self.p1score >= self.goalScore:
            self.player = None
            self.winner = 1
            self.isOver = True
        elif self.p2score >= self.goalScore:
            self.player = None
            self.winner = 2
            self.isOver = True

        #If verbose, print the game state. OR, do this job somewhere else. I'm not sure.
        if verbose:
            self.printState()

        return score


    def printRowOfLevers(self, rowIndex):  #Helper function of printState
        #Print 1st line
        print(" " * (self.h-rowIndex)*2 + '|', end="")
        for lever in self.levers[rowIndex]:
            lever.printTop()
        print("")  #Jump to new line
        #Print 2nd line
        print(" " * (self.h-rowIndex)*2 + '|', end="")
        for lever in self.levers[rowIndex]:
            lever.printMiddle()
        print("")  #Jump to new line
        #Print 3rd line
        leftBottomEdge = " |" if rowIndex == self.h-1 else "/ "
        rightBottomEdge = "\b|" if rowIndex == self.h-1 else "\\"
        print(" " * ((self.h-rowIndex)*2-1) + leftBottomEdge, end="")
        for lever in self.levers[rowIndex]:
            lever.printBottom()

        print(rightBottomEdge)

    def printState(self):
        #Print score line
        print("Player 1                 Player 2")
        print("%s\t\t\t\t%s" % (self.p1score, self.p2score))
        #Print TURN/WINNER line
        if self.winner:
            if self.winner == 1:
                print("WINNER                           ")
            elif self.winner == 2:
                print("                           WINNER")
        else:
            if self.player == 1:
                print("TURN                             ")
            elif self.player == 2:
                print("                             TURN")
        #Print the available moves row
        print(' '*self.h*2+"|", end="")
        for i in range(self.w*2):
            print(str(i)+"|", end="")
        print()
        #Print enterance row
        print(' '*self.h*2 + "|" + " |"*self.w*2)
        #Print intermediary rows
        for idx, _ in enumerate(self.levers):
            self.printRowOfLevers(idx)
        #Print exit row
        print('  ', end="")
        print("|" + " |"*(self.w+self.h-1)*2)
        #Print buckets
        print("  |", end="")
        for b in self.buckets:
            numberToPrint = b//10
            characterToPrint = " " if numberToPrint == 0 else str(numberToPrint)
            print(characterToPrint+"|", end="")
        print()  #In order to move on to the next line
        print("  |", end="")
        for b in self.buckets:
            print(str(b%10)+"|", end="")
        print()
        print()

    def getState(self):  #state returns as a string
        state = []
        for r in range(len(self.levers)):
            for l in self.levers[r]:
                state.append(l.position)
                state.append(l.coin)
        return str(state), self.w, self.h

    def setState(self, state, w, h):  #adjust the states of levers according to the given state
        if self.w != w or self.h != h:
            print(self.w)
            print(w)
            print(self.h)
            print(h)
            raise(Exception("width and height not compatible for setState()."))

        s = list(state)
        for r in range(len(self.levers)):
            for l in self.levers[r]:
                l.position = s.pop(0)
                l.coin = s.pop(0)

    def nextStatesAndRewards(self):  #Returns next states as strings and associated rewards.
        currentState = self.getState()  #preserving the current state
        nextStatesAndRewards = []
        #because nextState[1] is width and nextState[2] is height
        for i in range(self.w*2):
            td = TreasureDrop(w=self.w, h=self.h)
            td.setState(currentState[0], currentState[1], currentState[2])
            reward = td.dropCoin(i, verbose=False)
            nextState = td.getState()[0]
            nextStatesAndRewards.append((nextState, reward))
        return nextStatesAndRewards

    def playable(self):
        return not self.isOver


## Play with Random Agent

In [5]:
td = TreasureDrop(w=4, h=5)

state = None
# Added variable definition of w
w = 4
while not td.isOver:
    reply = input("Make a move: ")
    if reply == "q":
        break
    move = int(reply)
    td.dropCoin(move, verbose=True)
    td.dropCoin(np.random.randint(low=0, high=2*w-1), verbose=True)  #The random Agent

Make a move: 1
Player 1                 Player 2
0				0
                             TURN
          |0|1|2|3|4|5|6|7|
          | | | | | | | | |
          |  O|  V|V  |  V|
          | / | / | \ | / |
         / /   /     \ /   \
        |  V|V  |V  |V  |V  |
        | / | \ | \ | \ | \ |
       / /     \   \   \   \ \
      |V  |  V|V  |  V|  V|V  |
      | \ | / | \ | / | / | \ |
     /   \ /     \ /   /     \ \
    |  V|  V|  V|  V|  V|V  |  V|
    | / | / | / | / | / | \ | / |
   / /   /   /   /   /     \ /   \
  |  V|V  |  V|V  |V  |V  |V  |  V|
  | / | \ | / | \ | \ | \ | \ | / |
  |/     \ /     \   \   \   \ /   |
  | | | | | | | | | | | | | | | | |
  |3|2|1| | | | | | | | | | |1|2|3|
  |4|1|3|8|5|3|2|1|1|2|3|5|8|3|1|4|

Player 1                 Player 2
0				0
TURN                             
          |0|1|2|3|4|5|6|7|
          | | | | | | | | |
          |  O|V  |V  |  V|
          | / | \ | \ | / |
         / /     \   \ /   \
        |  V|  V|V  |V  |V  |
        | / |

## 100 COIN/PONT = WINNER